**ETL Report (Data Extraction and Transformation Portion)** 

1. Import relevant packages and call Web API to get census data

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from config import key
aListCS = ['EMP','NAICS2017','NAICS2017_LABEL','PAYANN','FIRMPDEMP']
aListCB = ['EMP','NAICS2017','NAICS2017_LABEL','PAYANN','TECHUSE','TECHUSE_LABEL','FIRMPDEMP']
variablesCS = ','.join(aListCS)
variablesCB = ','.join(aListCB)
urlCS = f'https://api.census.gov/data/2018/abscs?get={variablesCS}&for=us:*&key={key}'
urlCB = f'https://api.census.gov/data/2018/abstcb?get={variablesCB}&for=us:*&key={key}'
responseCS = requests.get(urlCS).json()
responseCB = requests.get(urlCB).json()

2. Convert census data into pandas Dataframe

In [ ]:
censusData = pd.DataFrame(responseCS)
censusData.columns = censusData.iloc[0]
censusData = censusData[1:]
censusDataCB = pd.DataFrame(responseCB)
censusDataCB.columns = censusDataCB.iloc[0]
censusDataCB = censusDataCB[1:]

3. Merge two Dataframes together

In [ ]:
mergedCensus = censusData.merge(censusDataCB,how='inner',on='NAICS2017_LABEL')

4. Clean the Data

In [ ]:
mergedCensus = mergedCensus.loc[mergedCensus['NAICS2017_LABEL'] != 'Total for all sectors']
mergedCensus = mergedCensus.drop(['NAICS2017_x','NAICS2017_y','us_x','us_y','TECHUSE'],axis=1)
mergedCensus.rename(columns={'NAICS2017_LABEL':'Industry','PAYANN_y':'Annual Payroll','EMP_y':'Number of Employees','TECHUSE_LABEL':'Tech Level'\
    ,'EMP_x':'Total Number of Employees (Industry)', 'PAYANN_x':'Total Annual Payroll (Industry)','FIRMPDEMP_x':'Total Number of Firms','FIRMPDEMP_y':'Number of Firms'},inplace=True)
mergedCensus[['Technology Used','Level']] = mergedCensus['Tech Level'].str.split(': ',expand=True)
mergedCensus = mergedCensus.drop('Tech Level', axis=1)
mergedCensus = mergedCensus.loc[~mergedCensus['Level'].isin(['Total use','Total Reporting',"Don't know"])]
mergedCensus.loc[mergedCensus['Level'] == 'Tested, but did not use in production or service','Level'] = 'Tested'

5. Transform the data

In [ ]:
numberList = ['Number of Employees','Annual Payroll','Total Number of Employees (Industry)','Total Annual Payroll (Industry)','Number of Firms','Total Number of Firms']
for i in numberList:
    mergedCensus[i] = pd.to_numeric(mergedCensus[i])

6. Make new calculated columns for visualizations

In [ ]:
mergedCensus['Percentage of Firms'] = mergedCensus['Number of Firms']/mergedCensus['Total Number of Firms']*100
mergedCensus['Average Salary'] = mergedCensus['Annual Payroll']/mergedCensus['Number of Employees'] 

**Visualizations**

Make a new Dataframe for summary data

In [ ]:
industry = pd.DataFrame({'Industry':mergedCensus['Industry'].unique(),'Total Annual Payroll (Industry)':\
    mergedCensus['Total Annual Payroll (Industry)'].unique(),'Number of Employees':mergedCensus['Total Number of Employees (Industry)'].unique()})
industry['Average Annual Salary'] = industry['Total Annual Payroll (Industry)']/industry['Number of Employees']

Set some common theme  and functions for all visualizations

In [ ]:
size = (18,10)
sns.set_theme(style="darkgrid")
def label_point(x, y, val, ax):
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x'], point['y'], str(point['val']))
PROPS = {
    'boxprops':{'facecolor':'none', 'edgecolor':'black'},
    'medianprops':{'color':'black'},
    'whiskerprops':{'color':'black'},
    'capprops':{'color':'black'}
}

Scatter plot for Total Annual Payroll and Total Number of Employees

In [ ]:
industry.plot.scatter(x='Number of Employees',y='Total Annual Payroll (Industry)', figsize=size, s = 300, alpha = 0.5)
plt.xlabel('Total Number of Employees (in tens of millions)', fontsize=16)
plt.ylabel('Total Annual Payroll (in hundred of million of dollars)', fontsize=16)
for i in industry.index:
    plt.text(industry['Number of Employees'][i],industry['Total Annual Payroll (Industry)'][i],industry['Industry'][i])
plt.title('Total Annual Payroll vs Total Number of Employees by Industry',fontsize=20)
plt.show()

Barplot on Top 10 Average Annual Salary by Industry

In [ ]:
plt.figure(figsize=size)
sortedHighIndustry = industry.sort_values(by=['Average Annual Salary'], ascending=False).head(10)
plt.title('Top 10 Average Salary by Industry',fontsize=20)
plt.xticks(rotation=45,ha='right')
ax = sns.barplot(x='Industry',y='Average Annual Salary',data=sortedHighIndustry, color='#2976bb')
plt.xlabel('Industry', fontsize=16)
plt.ylabel('Average Salary (in thousand of dollars)', fontsize=16)
for container in ax.containers:
    ax.bar_label(container)
plt.show()

Barplot on Bot 10 Average Annual Salary by Industry

In [ ]:
plt.figure(figsize=size)
sortedLowIndustry = industry.sort_values(by=['Average Annual Salary']).head(10)
plt.title('Bottom 10 Average Salary by Industry',fontsize=20)
plt.xticks(rotation=45,ha='right')
ax = sns.barplot(x='Industry',y='Average Annual Salary',data=sortedLowIndustry, color='#2976bb')
plt.xlabel('Industry', fontsize=16)
plt.ylabel('Average Salary (in thousand of dollars)', fontsize=16)
for container in ax.containers:
    ax.bar_label(container)
plt.show()

Set Subset for AI and Cloud-Based Technology

In [ ]:
ai = mergedCensus.loc[mergedCensus['Technology Used']=='Artificial Intelligence']
cb = mergedCensus.loc[mergedCensus['Technology Used']=='Cloud-Based']

Boxplot for Average Salary of Industries by Level of Artificial Intelligence Used 

In [ ]:
plt.figure(figsize=size)
ax = sns.boxplot(x='Level',y='Average Salary',data=ai, order=['Did not use','Tested','Low use','Moderate use','High use'], **PROPS)
plt.xlabel('Tech Level', fontsize=16)
plt.ylabel('Average Annual Salary (in thousand of dollars)', fontsize=16)
plt.title('Average Salary of Industries by Level of Artificial Intelligence Used',fontsize=20)
plt.show()

Boxplot for Average Salary of Industries by Level of Cloud-Based Technology Used 

In [ ]:
plt.figure(figsize=size)
ax = sns.boxplot(x='Level',y='Average Salary',data=cb, order=['Did not use','Tested','Low use','Moderate use','High use'],**PROPS)
plt.xlabel('Level of Use', fontsize=16)
plt.ylabel('Average Annual Salary (in thousand of dollars)', fontsize=16)
plt.title('Average Salary of Industries by Level of Cloud-Based Technology Used',fontsize=20)
plt.show()

Observe certain level of use in technology by filtering the dataframe

In [ ]:
industryAverage = industry[['Industry','Average Annual Salary']]
highUseAI = ai.loc[ai['Level']=='High use']
highUseAI = highUseAI.loc[highUseAI['Percentage of Firms'] > 0]

noUseAI = ai.loc[ai['Level']=='Did not use']
noUseAI = noUseAI.loc[noUseAI['Percentage of Firms'] > 0]

testUseAI = ai.loc[ai['Level']=='Tested']
testUseAI = testUseAI.loc[testUseAI['Percentage of Firms'] > 0]

highUseCloud = cb.loc[cb['Level']=='High use']
highUseCloud = highUseCloud.loc[highUseCloud['Percentage of Firms'] > 0]

noUseCloud = cb.loc[cb['Level']=='Did not use']
noUseCloud = noUseCloud.loc[noUseCloud['Percentage of Firms'] > 0]

testUseCloud = cb.loc[cb['Level']=='Tested']
testUseCloud = testUseCloud.loc[testUseCloud['Percentage of Firms'] > 0]

Bar Chart on Top 10 Industry with Highest Percentage of Firms Uses A High Level of AI

In [ ]:
# Sort 
sortedHighUseAI = highUseAI.sort_values(by=['Percentage of Firms'], ascending=False).head(10)
sortedHighUseAI = sortedHighUseAI.merge(industryAverage,how='inner',on='Industry')
ax = sortedHighUseAI.plot.bar(x='Industry',y='Percentage of Firms',rot=45,figsize=size,legend=False)
# Add Number over each bar
for container in ax.containers:
    ax.bar_label(container)
plt.title('Top 10 Industry with Highest Percentage of Firms Uses A High Level of AI',fontsize=20)
plt.xlabel('Industry',fontsize=16)
plt.xticks(ha='right')
plt.ylabel('Percentage of Firms',fontsize=16)
plt.show()

Clustered Bar Chart on the Difference in Average Annual Salary with High Use of AI

In [ ]:
ax = sortedHighUseAI.plot(x='Industry',y=['Average Annual Salary','Average Salary'], kind='bar', figsize=size,rot = 45)
ax.legend(['Industry Average','Industry Average with A High Level of AI'])
plt.xticks(ha='right')
plt.xlabel('industry', fontsize=16)
plt.ylabel('Average Annual Salary (in Thousand of Dollars)',fontsize=16)
plt.title('Comparison on Average Annual Salary with a High Level Usage of AI', fontsize = 20)
plt.show()

Bar Chart on Top 10 Industry with Highest Percentage of Firms Do not Use AI

In [ ]:
# Sort 
sortedNoUseAI = noUseAI.sort_values(by=['Percentage of Firms'], ascending=False).head(10)
sortedNoUseAI = sortedNoUseAI.merge(industryAverage,how='inner',on='Industry')
ax = sortedNoUseAI.plot.bar(x='Industry',y='Percentage of Firms',rot=45,figsize=size,legend=False)
# Add Number over each bar
for container in ax.containers:
    ax.bar_label(container)
plt.title('Top 10 Industry with Highest Percentage of Firms Do not Use AI',fontsize=20)
plt.xlabel('Industry',fontsize=16)
plt.xticks(ha='right')
plt.ylabel('Percentage of Firms',fontsize=16)
plt.show()

Clustered Bar Chart on the Difference in Average Annual Salary with No Use of AI

In [ ]:
ax = sortedNoUseAI.plot(x='Industry',y=['Average Annual Salary','Average Salary'], kind='bar', figsize=size,rot = 45)
ax.legend(['Industry Average','Industry Average with No Usage of AI'])
plt.xticks(ha='right')
plt.xlabel('industry', fontsize=16)
plt.ylabel('Average Annual Salary (in Thousand of Dollars)',fontsize=16)
plt.title('Comparison on Average Annual Salary with No Usage of AI', fontsize = 20)
plt.show()

Bar Chart on Top 10 Industry with Highest Percentage of Firms Tested AI

In [ ]:
# Sort 
sortedTestUseAI = testUseAI.sort_values(by=['Percentage of Firms'], ascending=False).head(10)
sortedTestUseAI = sortedTestUseAI.merge(industryAverage,how='inner',on='Industry')
ax = sortedTestUseAI.plot.bar(x='Industry',y='Percentage of Firms',rot=45,figsize=size,legend=False)
# Add Number over each bar
for container in ax.containers:
    ax.bar_label(container)
plt.title('Top 10 Industry with Highest Percentage of Firms Tested AI',fontsize=20)
plt.xlabel('Industry',fontsize=16)
plt.xticks(ha='right')
plt.ylabel('Percentage of Firms',fontsize=16)
plt.show()

Clustered Bar Chart on the Difference in Average Annual Salary with Tested AI

In [ ]:
ax = sortedTestUseAI.plot(x='Industry',y=['Average Annual Salary','Average Salary'], kind='bar', figsize=size,rot = 45)
ax.legend(['Industry Average','Industry Average with Tested AI'])
plt.xticks(ha='right')
plt.xlabel('industry', fontsize=16)
plt.ylabel('Average Annual Salary (in Thousand of Dollars)',fontsize=16)
plt.title('Comparison on Average Annual Salary with Tested AI', fontsize = 20)
plt.show()

Bar Chart on Top 10 Industry with Highest Percentage of Firms Uses A High Level of Cloud-Based Technology

In [ ]:
# Sort 
sortedHighUseCloud = highUseCloud.sort_values(by=['Percentage of Firms'], ascending=False).head(10)
sortedHighUseCloud = sortedHighUseCloud.merge(industryAverage,how='inner',on='Industry')
ax = sortedHighUseCloud.plot.bar(x='Industry',y='Percentage of Firms',rot=45,figsize=size,legend=False)
# Add Number over each bar
for container in ax.containers:
    ax.bar_label(container)
plt.title('Top 10 Industry with Highest Percentage of Firms Uses A High Level of Cloud-Based Technology',fontsize=20)
plt.xlabel('Industry',fontsize=16)
plt.xticks(ha='right')
plt.ylabel('Percentage of Firms',fontsize=16)
plt.show()

Clustered Bar Chart on the Difference in Average Annual Salary with High Use of Cloud-Based Technology

In [ ]:
ax = sortedHighUseCloud.plot(x='Industry',y=['Average Annual Salary','Average Salary'], kind='bar', figsize=size,rot = 45)
ax.legend(['Industry Average','Industry Average with A High Level of Cloud'])
plt.xticks(ha='right')
plt.xlabel('industry', fontsize=16)
plt.ylabel('Average Annual Salary (in Thousand of Dollars)',fontsize=16)
plt.title('Comparison on Average Annual Salary with a High Level Usage of Cloud-Based Technology', fontsize = 20)
plt.show()

Bar Chart on Top 10 Industry with Highest Percentage of Firms Do not Use Cloud Based Technology

In [ ]:
# Sort 
sortedNoUseCloud = noUseCloud.sort_values(by=['Percentage of Firms'], ascending=False).head(10)
sortedNoUseCloud = sortedNoUseCloud.merge(industryAverage,how='inner',on='Industry')
ax = sortedNoUseCloud.plot.bar(x='Industry',y='Percentage of Firms',rot=45,figsize=size,legend=False)
# Add Number over each bar
for container in ax.containers:
    ax.bar_label(container)
plt.title('Top 10 Industry with Highest Percentage of Firms Do not Use Cloud-Based Technology',fontsize=20)
plt.xlabel('Industry',fontsize=16)
plt.xticks(ha='right')
plt.ylabel('Percentage of Firms',fontsize=16)
plt.show()

Clustered Bar Chart on the Difference in Average Annual Salary with No Use of Cloud-Based Technology

In [ ]:
ax = sortedNoUseCloud.plot(x='Industry',y=['Average Annual Salary','Average Salary'], kind='bar', figsize=size,rot = 45)
ax.legend(['Industry Average','Industry Average with No Use of Cloud'])
plt.xticks(ha='right')
plt.xlabel('industry', fontsize=16)
plt.ylabel('Average Annual Salary (in Thousand of Dollars)',fontsize=16)
plt.title('Comparison on Average Annual Salary with No Usage of Cloud-Based Technology', fontsize = 20)
plt.show()

Bar Chart on Top 10 Industry with Highest Percentage of Firms Tested Cloud-Based Technology

In [ ]:
# Sort 
sortedTestUseCloud = testUseCloud.sort_values(by=['Percentage of Firms'], ascending=False).head(10)
sortedTestUseCloud = sortedTestUseCloud.merge(industryAverage,how='inner',on='Industry')
ax = sortedTestUseCloud.plot.bar(x='Industry',y='Percentage of Firms',rot=45,figsize=size,legend=False)
# Add Number over each bar
for container in ax.containers:
    ax.bar_label(container)
plt.title('Top 10 Industry with Highest Percentage of Firms Tested Cloud-Based Technology',fontsize=20)
plt.xlabel('Industry',fontsize=16)
plt.xticks(ha='right')
plt.ylabel('Percentage of Firms',fontsize=16)
plt.show()

Clustered Bar Chart on the Difference in Average Annual Salary with Tested of Cloud-Based Technology

In [ ]:
ax = sortedTestUseCloud.plot(x='Industry',y=['Average Annual Salary','Average Salary'], kind='bar', figsize=size,rot = 45)
ax.legend(['Industry Average','Industry Average with Tested Cloud'])
plt.xticks(ha='right')
plt.xlabel('industry', fontsize=16)
plt.ylabel('Average Annual Salary (in Thousand of Dollars)',fontsize=16)
plt.title('Comparison on Average Annual Salary with Tested Cloud-Based Technology', fontsize = 20)
plt.show()